(Описание к коду написано под элементами с кодом)

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("..\\data\\inputs"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

..\data\inputs\test.csv
..\data\inputs\train.csv


Смотрим какие файлы есть в папке

In [12]:
train_data = pd.read_csv("..\\data\\inputs\\train.csv")
test_data = pd.read_csv("..\\data\\inputs\\test.csv")

train_data.head()
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


Загружаем тренировочные и тестовые данные из файлов

In [19]:
X = train_data.drop(columns = 'Transported')
y = train_data['Transported']

Тренировочные данные делим на две части, где одна - только информация о пассажире, а другая - был ли он транспортирован

In [20]:
def fill_missing_values(data):
    numeric_data = [column for column in data.select_dtypes(["int", "float"])]
    categorical_data = [column for column in data.select_dtypes(exclude=["int", "float"])]

    for col in numeric_data:
        data[col] = data[col].infer_objects(copy=False).fillna(data[col].median())

    for col in categorical_data:
        data[col] = data[col].fillna(data[col].value_counts().index[0]).infer_objects(copy=False)

    return data


def data_preparations(data):
    data = fill_missing_values(data)

    cabin_data = data["Cabin"].str.split("/", expand=True)
    cabin_data.columns = ["Deck", "Num", "Side"]
    cabin_data["Num"] = cabin_data["Num"].fillna(-1).astype(int)
    cabin_data["Deck"] = cabin_data["Deck"].fillna("Unknown")
    cabin_data["Side"] = cabin_data["Side"].fillna("Unknown")

    data["CryoSleep"] = data["CryoSleep"].astype(bool).fillna(False).astype(int)
    data["VIP"] = data["VIP"].astype(bool).fillna(False).astype(int)

    # Пробую с суммой всех трат
    spends_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    data["Spends"] = data[spends_columns].sum(axis=1)

    data["NoSpends"] = (data["Spends"] == 0)
    # print(data.head())

    numeric_columns = [
        'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', "Spends",
        "NoSpends", "Age", "CryoSleep", "VIP"
    ]
    numerics = data.copy()[numeric_columns]

    columns_for_dummies = ["HomePlanet", "Destination"]
    dummies = pd.get_dummies(cabin_data[["Deck", "Side"]].join(data.copy()[columns_for_dummies]))

    result = pd.concat([numerics, dummies], axis=1)

    # print(len(result.columns))

    return result

X = data_preparations(train_data)

C:\Users\IIsovaII\AppData\Local\Temp\ipykernel_15708\517489211.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].fillna(data[col].value_counts().index[0]).infer_objects(copy=False)
C:\Users\IIsovaII\AppData\Local\Temp\ipykernel_15708\517489211.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].fillna(data[col].value_counts().index[0]).infer_objects(copy=False)


Добавляем функции обработци данных

In [29]:
import optuna
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_breast_cancer


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

def objective_xgb(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'n_estimators': 300,
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    model = XGBClassifier(**param)
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)
    return auc


def objective_lgbm(trial):
    param = {
                "objective": "binary",
                "metric": "",
                "boosting_type": 'gbdt',
                "n_estimators": 300,
                "learning_rate": 0.043,
                "num_leaves": trial.suggest_int("num_leaves", 20, 150),
                "max_depth": trial.suggest_int("max_depth", 3, 5, step=1),
                "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
                "subsample": trial.suggest_float("subsample", 0.4, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 12.0, log=True),
                "random_state": trial.suggest_int("random_state", 2, 100, step=2),
                "n_jobs": -1,
                "verbose": -1
            }

    model = LGBMClassifier(**param)
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)
    return auc


def objective_catboost(trial):
    param = {
        'loss_function': 'Logloss',
        'iterations': 300,
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 1e-3, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True)
    }

    model = CatBoostClassifier(**param, verbose=0)
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)
    return auc

study_xgb = optuna.create_study(direction='maximize')
study_lgbm = optuna.create_study(direction='maximize')
study_catboost = optuna.create_study(direction='maximize')

study_xgb.optimize(objective_xgb, n_trials=3)
study_lgbm.optimize(objective_lgbm, n_trials=3)
study_catboost.optimize(objective_catboost, n_trials=3)

best_trial_xgb = study_xgb.best_trial.params
best_trial_lgbm = study_lgbm.best_trial.params
best_trial_catboost = study_catboost.best_trial.params

print('Best trial for XGBoost:', study_xgb.best_trial.params)
print('Best trial for LightGBM:', study_lgbm.best_trial.params)
print('Best trial for CatBoost:', study_catboost.best_trial.params)

[I 2025-03-19 05:40:39,246] A new study created in memory with name: no-name-11da657c-34fa-4ee7-8d3b-e5c5525a49dd
[I 2025-03-19 05:40:39,247] A new study created in memory with name: no-name-6e54bcc7-1cc4-4d2b-9b10-80f161cb5952
[I 2025-03-19 05:40:39,248] A new study created in memory with name: no-name-ea481118-cd0a-49a3-be20-e620aa540fe8
[I 2025-03-19 05:40:39,528] Trial 0 finished with value: 0.869081351080351 and parameters: {'lambda': 6.450657006879346, 'alpha': 0.09967016644802659, 'learning_rate': 0.0016360313565073196, 'max_depth': 7, 'min_child_weight': 8, 'gamma': 0.013364473811763538, 'subsample': 0.7210887024994775, 'colsample_bytree': 0.9975679265157434}. Best is trial 0 with value: 0.869081351080351.
[I 2025-03-19 05:40:39,787] Trial 1 finished with value: 0.8846734871514026 and parameters: {'lambda': 4.511010016261137, 'alpha': 0.0027058196803264813, 'learning_rate': 0.016307762997959118, 'max_depth': 6, 'min_child_weight': 5, 'gamma': 0.6448598275680949, 'subsample': 0.

Best trial for XGBoost: {'lambda': 0.8535798742328009, 'alpha': 0.007058387365648772, 'learning_rate': 0.026655561164108485, 'max_depth': 5, 'min_child_weight': 6, 'gamma': 1.3263839306807987, 'subsample': 0.8000194798947529, 'colsample_bytree': 0.8555485087762065}
Best trial for LightGBM: {'num_leaves': 53, 'max_depth': 5, 'min_child_samples': 52, 'subsample': 0.42827940525575325, 'colsample_bytree': 0.627683725634276, 'reg_alpha': 0.015389360291594113, 'reg_lambda': 2.074982694514809, 'random_state': 50}
Best trial for CatBoost: {'depth': 4, 'learning_rate': 0.030832390985637493, 'random_strength': 0.036324258617002246, 'bagging_temperature': 0.08039473844291635, 'border_count': 175, 'l2_leaf_reg': 4.8766734778419885}


Прописывам метрики для трех алгоритмов, которые потенциально должны хорошо подойти для решения данной задачи. Алгоритмы: XGBClassifier, LGBMClassifier, CatBoostClassifier. У каждого есть как возможность внутреннего обучения, так и подключен модуль optuna, для перебора лучших параметров для алгоритмов, с целью повышения эффективности.

In [30]:
best_xgb = XGBClassifier(**best_trial_xgb)
best_xgb.fit(X_train, y_train)
auc_xgb = roc_auc_score(y_valid, best_xgb.predict_proba(X_valid)[:, 1])

best_lgbm = LGBMClassifier(**best_trial_lgbm)
best_lgbm.fit(X_train, y_train)
auc_lgbm = roc_auc_score(y_valid, best_lgbm.predict_proba(X_valid)[:, 1])

best_catboost = CatBoostClassifier(**best_trial_catboost, verbose=0)
best_catboost.fit(X_train, y_train)
auc_catboost = roc_auc_score(y_valid, best_catboost.predict_proba(X_valid)[:, 1])

print(f'Best AUC for XGBoost: {auc_xgb:.4f}')
print(f'Best AUC for LightGBM: {auc_lgbm:.4f}')
print(f'Best AUC for CatBoost: {auc_catboost:.4f}')

Best AUC for XGBoost: 0.8812
Best AUC for LightGBM: 0.8835
Best AUC for CatBoost: 0.8936


Сравниваем лучшие результаты алгоритмов. Наибольшее значение получилось у CatBoostClassifier.

In [31]:
X_test = data_preparations(test_data)

best_predictions = best_catboost.predict_proba(X_test)[:, 1]
best_predictions = (best_predictions > 0.5).astype(int)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': best_predictions})
output["Transported"] = output["Transported"].astype(bool)

output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

Submission was successfully saved!


Обучаем итоговую модель на алгоритме CatBoostClassifier. Записываем результат в файл submission.csv

In [32]:
result = pd.read_csv("submission.csv")
result.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
